**Dependencies**

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Reshape
from keras import layer
import cv2s
import matplotlib.pyplot as plt
import os 
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

**Data Preprocessing**

In [2]:
x=[]
y=[]
mapping={
    "anger":0,
    "disgust":1,
    "fear":2,
    "happy":3,
    "neutral":4
}
path=r"C:\Users\Suma\Desktop\FACE_RECOGNITION\Emotion"
for relate in mapping.keys():
    for img in os.listdir(os.path.join(path,relate)):
        im=cv2.imread(os.path.join(path,relate,img))
        im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im=cv2.resize(im,(40,40))
        x.append(im)
        y.append(mapping[relate])
x=np.array(x)
y=np.array(y)
x=x.reshape(x.shape[0],40,40,1)
x=x.astype('float32')
x=x/255
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train,test_size=0.1)
early_stopping=EarlyStopping(
    patience=15, 
    restore_best_weights=True,
)

**CNN Model Architecture**

In [3]:
emotion =Sequential()
emotion.add(Conv2D(filters=150,kernel_size=3,strides=(1,1),padding="valid",input_shape=(40,40,1),activation="relu"))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization())
emotion.add(Conv2D(filters=80,kernel_size=3,strides=(1,1),padding="same")) 
emotion.add(layers.LeakyReLU(alpha=0.1))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization())
emotion.add(Conv2D(filters=10,kernel_size=5,strides=(1,1),padding="valid")) 
emotion.add(layers.LeakyReLU(alpha=0.1))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization())
emotion.add(Flatten())
emotion.add(layers.Dropout(0.4))
emotion.add(Dense(60,activation="relu"))
emotion.add(Dense(5,activation="softmax"))
emotion.summary()
emotion.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
emotion.fit(x_train,y_train,epochs=200,callbacks=[early_stopping],validation_data=(x_valid, y_valid)) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 150)       1500      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 150)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 19, 19, 150)       600       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 80)        108080    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 19, 19, 80)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 80)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 80)          3

In [4]:
print(emotion.evaluate(x_test,y_test))

12/12 [==============================] - 1s 77ms/step - loss: 1.0725 - accuracy: 0.5916
[1.0725431442260742, 0.5916230082511902]


In [5]:
emotion.save("emotion_model.h5")

In [6]:
new_model=keras.models.load_model("emotion_model.h5")

In [7]:
new_model.predict(x_valid[0].reshape(-1,40,40,1))

array([[6.8662310e-01, 1.3794465e-01, 1.7910822e-04, 1.7522255e-01,
        3.0659077e-05]], dtype=float32)